In [43]:
from deepcave.runs.converters.smac3v2 import SMAC3v2Run
from deepcave.plugins.hyperparameter.importances import Importances
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [44]:
def evaluate_feature_importance(path_to_smac_run:str, objective_id:int):
    """
    Code adapted from https://github.com/automl/DeepCAVE/blob/main/examples/api/importances.py
    """
    run = SMAC3v2Run.from_path(path_to_smac_run)
    objective_id = run.get_objective_ids()[objective_id]
    budget_ids = run.get_budget_ids()

    # Instantiate the plugin
    plugin = Importances()
    inputs = plugin.generate_inputs(
        hyperparameter_names=list(run.configspace.keys()),
        objective_id=objective_id,
        budget_ids=budget_ids,
        method="global",
        n_hps=20,
        n_trees=10,
    )
    # Note: Filter variables are not considered.
    outputs = plugin.generate_outputs(run, inputs)

    return outputs

In [45]:
outputs = evaluate_feature_importance("smac3_output/grow_once/grow_with_once_noise_in_configspace/0", 0)[0]

In [46]:
outputs = sorted(outputs.items(), key=lambda x: x[1][0], reverse=True)
outputs

[('learning_rate',
  (0.6845189583627619,
   0.05868178611778061,
   0.6845189583627619,
   0.05868178611778061)),
 ('ent_coef',
  (0.07274040022738897,
   0.016264706096086247,
   0.07274040022738897,
   0.016264706096086247)),
 ('noise_level',
  (0.03306886472718433,
   0.0016551818703481745,
   0.03306886472718433,
   0.0016551818703481745)),
 ('n_epochs',
  (0.029553324759990784,
   0.006946218973257863,
   0.029553324759990784,
   0.006946218973257863)),
 ('momentum_reset',
  (0.014707840781433789,
   0.0008146347915968107,
   0.014707840781433789,
   0.0008146347915968107)),
 ('batch_size',
  (0.004650269570486437,
   0.00012540615961578725,
   0.004650269570486437,
   0.00012540615961578725))]

In [47]:
importances = [item[1][0] for item in outputs]
errors = [item[1][1] for item in outputs]
features = ["Learning Rate", "Entropy Coefficient", "Noise Level", "Batch Size", "n Epochs", "Momentum Reset"]

cmap = plt.get_cmap("tab10")  # Get the tab10 colormap
colors = cmap(np.arange(len(features))) 

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(features, importances, color=colors, yerr=errors, capsize=15, error_kw={'elinewidth': 2, 'capthick': 2, 'ecolor': 'black'})
ax.set_xlabel("Features")
ax.set_ylabel("Importance")
ax.set_title("Feature Importances; Grow Width Once", fontsize=18, fontweight="bold")
ax.legend(title="Model Type", fontsize=12, title_fontsize=14, loc="center", bbox_to_anchor=(0.5, -0.16), ncol=3)
ax.grid(True, linestyle="--", alpha=0.7)

plt.savefig("plotting/minihack/grow_width_once/grow_width_once_feature_importances.png", bbox_inches='tight')

/tmp/ipykernel_385882/2126500359.py:14: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(title="Model Type", fontsize=12, title_fontsize=14, loc="center", bbox_to_anchor=(0.5, -0.16), ncol=3)
